In [1]:
import tensorflow as tf


/root/anaconda2/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda2/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda2/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
# tf.__path__

['/root/anaconda2/envs/py35/lib/python3.5/site-packages/tensorflow']

In [4]:
import os
import numpy as np
import tensorflow as tf
import sys
vggdir = os.getcwd() + '/' + 'tensorflow_vgg'
sys.path.append(vggdir)
from tensorflow_vgg import vgg16
from tensorflow_vgg import utils

/root/anaconda2/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
# data_dir = 'zsy_all/'
# data_dir = 'zsy_train/'
data_dir = 'ori_zsy/'
# data_dir = 'all_5_data/'
contents = os.listdir(data_dir)
classes = [each for each in contents if os.path.isdir(data_dir + each)]

In [6]:
# print (contents)
# print(classes)

### 使用vgg16模型来处理图片，生成结果为vgg16处理后的图片特征值
### codes.shape = (2095, 4096)

In [7]:
# 首先设置计算batch的值，如果运算平台的内存越大，这个值可以设置得越高
batch_size = 32   #  32   64    128  
# 用codes_list来存储特征值
codes_list = []
# 用labels来存储类别
labels = []
# batch数组用来临时存储图片数据
batch = []

codes = None

with tf.Session() as sess:
    # 构建VGG16模型对象
    vgg = vgg16.Vgg16()
#     print ("vgg -->", vgg)
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3], name='input_')
    with tf.name_scope("content_vgg"):
        # 载入VGG16模型
        vgg.build(input_)
    
    print('input_.graph =   {0}'.format(input_.graph))
    
    
    # 对每个不同种类的蔬菜分别用VGG16计算特征值
    for each in classes:
        print("Starting {} images".format(each))
        class_path = data_dir + each
        files = os.listdir(class_path)
        for ii, file in enumerate(files, 1):
            # 载入图片并放入batch数组中
            img = utils.load_image(os.path.join(class_path, file))
#             print (os.path.join(class_path, file))
#             print (img)
            
            batch.append(img.reshape((1, 224, 224, 3)))
            labels.append(each)
            
            # 如果图片数量到了batch_size则开始具体的运算
            if ii % batch_size == 0 or ii == len(files):
                images = np.concatenate(batch)
                
#                 print(images.shape) 
#                 assert images.shape == (16, 224, 224, 3)
                
                feed_dict = {input_: images}
                # 计算特征值
                codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)
                
                # 将结果放入到codes数组中
                if codes is None:
                    codes = codes_batch
                else:
                    codes = np.concatenate((codes, codes_batch))
                
                # 清空数组准备下一个batch的计算
                batch = []
                print('{} images processed'.format(ii))

/home/ces/tools/ws_andre/vege_transfer/tensorflow_vgg/vgg16.npy
npy file loaded
build model started
build model finished: 1s
input_.graph =   <tensorflow.python.framework.ops.Graph object at 0x7f54b7e63978>
Starting qingcai images


/root/anaconda2/envs/py35/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/root/anaconda2/envs/py35/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


32 images processed
64 images processed
96 images processed
128 images processed
160 images processed
192 images processed
224 images processed
256 images processed
288 images processed
320 images processed
352 images processed
384 images processed
416 images processed
448 images processed
480 images processed
512 images processed
544 images processed
574 images processed
Starting qc images
32 images processed
64 images processed
96 images processed
128 images processed
160 images processed
192 images processed
224 images processed
256 images processed
288 images processed
320 images processed
352 images processed
384 images processed
416 images processed
448 images processed
480 images processed
512 images processed
544 images processed
576 images processed
608 images processed
640 images processed
656 images processed
Starting jmc images
32 images processed
64 images processed
96 images processed
128 images processed
160 images processed
192 images processed
224 images processed
256 

In [8]:
with open('codes', 'w') as f:
    codes.tofile(f)
    
import csv
with open('labels', 'w') as f:
    writer = csv.writer(f, delimiter='\n')
    writer.writerow(labels)

In [9]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(labels)

labels_vecs = lb.transform(labels) # one-hot

/root/anaconda2/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda2/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [10]:
# print (labels)
# print (labels_vecs)

In [11]:
# assert codes.shape == (2095, 4096) # 意味着 图像经过vgg16预处理为4096长度的高级抽象的图像特征
# # labels.count('jimaocai') == 773
# assert labels_vecs.shape == (2095, 5)

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

ss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
train_idx, val_idx = next(ss.split(codes, labels))

half_val_len = int(len(val_idx)/2)
val_idx, test_idx = val_idx[:half_val_len], val_idx[half_val_len:]

train_x, train_y = codes[train_idx], labels_vecs[train_idx]
val_x, val_y = codes[val_idx], labels_vecs[val_idx]
test_x, test_y = codes[test_idx], labels_vecs[test_idx]

print("Train shapes (x, y):", train_x.shape, train_y.shape)
print("Validation shapes (x, y):", val_x.shape, val_y.shape)
print("Test shapes (x, y):", test_x.shape, test_y.shape)

Train shapes (x, y): (1508, 4096) (1508, 3)
Validation shapes (x, y): (189, 4096) (189, 3)
Test shapes (x, y): (189, 4096) (189, 3)


/root/anaconda2/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [13]:
from tensorflow.python.framework import graph_util
import shutil
output_graph = 'checkpoints/zsy.pb'

In [14]:
def get_batches(x, y, n_batches=32):
    """ 这是一个生成器函数，按照n_batches的大小将数据划分了小块 """
    batch_size = len(x) // n_batches
    
    for ii in range(0, n_batches*batch_size, batch_size):
        # 如果不是最后一个batch，那么这个batch中应该有batch_size个数据
        if ii != (n_batches-1)*batch_size:
            X, Y = x[ii: ii+batch_size], y[ii: ii+batch_size] 
        # 否则的话，那剩余的不够batch_size的数据都凑入到一个batch中
        else:
            X, Y = x[ii:], y[ii:]
        # 生成器语法，返回X和Y
        yield X, Y

In [15]:
# 输入数据的维度
inputs_fea = tf.placeholder(tf.float32, shape=[None, codes.shape[1]], name='inputs_fea')
# 标签数据的维度
labels_ = tf.placeholder(tf.int64, shape=[None, labels_vecs.shape[1]])

# 加入一个256维的全连接的层
fc = tf.contrib.layers.fully_connected(inputs_fea, 256)

# 加入一个  label_vecs列数 维的全连接层
logits = tf.contrib.layers.fully_connected(fc, labels_vecs.shape[1], activation_fn=None)

# 计算cross entropy值  新版API:softmax_cross_entropy_with_logits_v2
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels_, logits=logits)

# 计算损失函数
cost = tf.reduce_mean(cross_entropy)

# 采用用得最广泛的AdamOptimizer优化器
optimizer = tf.train.AdamOptimizer().minimize(cost)

# 得到最后的预测分布
predicted = tf.nn.softmax(logits, name='predicted')

# print (predicted)

# 计算准确度
correct_pred = tf.equal(tf.argmax(predicted, 1), tf.argmax(labels_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

# 运行多少轮次
epochs = 30
# 统计训练效果的频率
iteration = 0
# 保存模型的保存器
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(epochs):
        for x, y in get_batches(train_x, train_y):
            feed = {inputs_fea: x,
                    labels_: y}
            # 训练模型
            loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            print("Epoch: {}/{}".format(e+1, epochs),
                  "Iteration: {}".format(iteration),
                  "Training loss: {:.5f}".format(loss))
            
            #  训练模型  add 
#             pred = sess.run(predicted, feed_dict=feed)
#             print ("pred :" , pred)
            
            iteration += 1

            if iteration % 3 == 0:
                feed = {inputs_fea: val_x,
                        labels_: val_y}
                val_acc = sess.run(accuracy, feed_dict=feed)
                # 输出用验证集验证训练进度
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Validation Acc: {:.4f}".format(val_acc))
    # 保存模型  为ckpt
    saver.save(sess, "checkpoints/zsy.ckpt")

    
    print('inputs_fea.graph =   {0}'.format(inputs_fea.graph))

    # 保存模型为pb文件
    graph_def = tf.get_default_graph().as_graph_def()
    output_graph_def = graph_util.convert_variables_to_constants(sess, graph_def,['predicted'])

    with tf.gfile.GFile(output_graph , 'wb') as f:
        f.write(output_graph_def.SerializeToString()) # 序列化输出


    print ("%d ops in the final graph." %len(output_graph_def.node))
    print (predicted)
    

Epoch: 1/30 Iteration: 0 Training loss: 5.80374
Epoch: 1/30 Iteration: 1 Training loss: 11.67281
Epoch: 1/30 Iteration: 2 Training loss: 16.29640
Epoch: 0/30 Iteration: 3 Validation Acc: 0.5556
Epoch: 1/30 Iteration: 3 Training loss: 4.55265
Epoch: 1/30 Iteration: 4 Training loss: 1.65550
Epoch: 1/30 Iteration: 5 Training loss: 1.47221
Epoch: 0/30 Iteration: 6 Validation Acc: 0.8413
Epoch: 1/30 Iteration: 6 Training loss: 1.04322
Epoch: 1/30 Iteration: 7 Training loss: 1.58252
Epoch: 1/30 Iteration: 8 Training loss: 1.31978
Epoch: 0/30 Iteration: 9 Validation Acc: 0.8677
Epoch: 1/30 Iteration: 9 Training loss: 1.43545
Epoch: 1/30 Iteration: 10 Training loss: 1.26352
Epoch: 1/30 Iteration: 11 Training loss: 1.50706
Epoch: 0/30 Iteration: 12 Validation Acc: 0.9577
Epoch: 1/30 Iteration: 12 Training loss: 0.34404
Epoch: 1/30 Iteration: 13 Training loss: 0.06668
Epoch: 1/30 Iteration: 14 Training loss: 0.15931
Epoch: 0/30 Iteration: 15 Validation Acc: 0.9788
Epoch: 1/30 Iteration: 15 Train

Epoch: 4/30 Iteration: 144 Validation Acc: 1.0000
Epoch: 5/30 Iteration: 144 Training loss: 0.00000
Epoch: 5/30 Iteration: 145 Training loss: 0.00001
Epoch: 5/30 Iteration: 146 Training loss: 0.00026
Epoch: 4/30 Iteration: 147 Validation Acc: 1.0000
Epoch: 5/30 Iteration: 147 Training loss: 0.00001
Epoch: 5/30 Iteration: 148 Training loss: 0.00000
Epoch: 5/30 Iteration: 149 Training loss: 0.00000
Epoch: 4/30 Iteration: 150 Validation Acc: 1.0000
Epoch: 5/30 Iteration: 150 Training loss: 0.00000
Epoch: 5/30 Iteration: 151 Training loss: 0.00037
Epoch: 5/30 Iteration: 152 Training loss: 0.00017
Epoch: 4/30 Iteration: 153 Validation Acc: 1.0000
Epoch: 5/30 Iteration: 153 Training loss: 0.00000
Epoch: 5/30 Iteration: 154 Training loss: 0.00000
Epoch: 5/30 Iteration: 155 Training loss: 0.00000
Epoch: 4/30 Iteration: 156 Validation Acc: 1.0000
Epoch: 5/30 Iteration: 156 Training loss: 0.00008
Epoch: 5/30 Iteration: 157 Training loss: 0.00025
Epoch: 5/30 Iteration: 158 Training loss: 0.00005


Epoch: 9/30 Iteration: 287 Training loss: 0.00015
Epoch: 8/30 Iteration: 288 Validation Acc: 1.0000
Epoch: 10/30 Iteration: 288 Training loss: 0.00000
Epoch: 10/30 Iteration: 289 Training loss: 0.00019
Epoch: 10/30 Iteration: 290 Training loss: 0.00001
Epoch: 9/30 Iteration: 291 Validation Acc: 1.0000
Epoch: 10/30 Iteration: 291 Training loss: 0.00001
Epoch: 10/30 Iteration: 292 Training loss: 0.00002
Epoch: 10/30 Iteration: 293 Training loss: 0.00001
Epoch: 9/30 Iteration: 294 Validation Acc: 1.0000
Epoch: 10/30 Iteration: 294 Training loss: 0.00001
Epoch: 10/30 Iteration: 295 Training loss: 0.00004
Epoch: 10/30 Iteration: 296 Training loss: 0.00000
Epoch: 9/30 Iteration: 297 Validation Acc: 1.0000
Epoch: 10/30 Iteration: 297 Training loss: 0.00000
Epoch: 10/30 Iteration: 298 Training loss: 0.00001
Epoch: 10/30 Iteration: 299 Training loss: 0.00000
Epoch: 9/30 Iteration: 300 Validation Acc: 1.0000
Epoch: 10/30 Iteration: 300 Training loss: 0.00001
Epoch: 10/30 Iteration: 301 Training 

Epoch: 14/30 Iteration: 424 Training loss: 0.00000
Epoch: 14/30 Iteration: 425 Training loss: 0.00000
Epoch: 13/30 Iteration: 426 Validation Acc: 1.0000
Epoch: 14/30 Iteration: 426 Training loss: 0.00001
Epoch: 14/30 Iteration: 427 Training loss: 0.00000
Epoch: 14/30 Iteration: 428 Training loss: 0.00001
Epoch: 13/30 Iteration: 429 Validation Acc: 1.0000
Epoch: 14/30 Iteration: 429 Training loss: 0.00000
Epoch: 14/30 Iteration: 430 Training loss: 0.00005
Epoch: 14/30 Iteration: 431 Training loss: 0.00008
Epoch: 13/30 Iteration: 432 Validation Acc: 1.0000
Epoch: 14/30 Iteration: 432 Training loss: 0.00000
Epoch: 14/30 Iteration: 433 Training loss: 0.00001
Epoch: 14/30 Iteration: 434 Training loss: 0.00005
Epoch: 13/30 Iteration: 435 Validation Acc: 1.0000
Epoch: 14/30 Iteration: 435 Training loss: 0.00001
Epoch: 14/30 Iteration: 436 Training loss: 0.00000
Epoch: 14/30 Iteration: 437 Training loss: 0.00000
Epoch: 13/30 Iteration: 438 Validation Acc: 1.0000
Epoch: 14/30 Iteration: 438 Tra

Epoch: 18/30 Iteration: 559 Training loss: 0.00006
Epoch: 18/30 Iteration: 560 Training loss: 0.00000
Epoch: 17/30 Iteration: 561 Validation Acc: 1.0000
Epoch: 18/30 Iteration: 561 Training loss: 0.00001
Epoch: 18/30 Iteration: 562 Training loss: 0.00004
Epoch: 18/30 Iteration: 563 Training loss: 0.00000
Epoch: 17/30 Iteration: 564 Validation Acc: 1.0000
Epoch: 18/30 Iteration: 564 Training loss: 0.00000
Epoch: 18/30 Iteration: 565 Training loss: 0.00000
Epoch: 18/30 Iteration: 566 Training loss: 0.00000
Epoch: 17/30 Iteration: 567 Validation Acc: 1.0000
Epoch: 18/30 Iteration: 567 Training loss: 0.00014
Epoch: 18/30 Iteration: 568 Training loss: 0.00007
Epoch: 18/30 Iteration: 569 Training loss: 0.00000
Epoch: 17/30 Iteration: 570 Validation Acc: 1.0000
Epoch: 18/30 Iteration: 570 Training loss: 0.00000
Epoch: 18/30 Iteration: 571 Training loss: 0.00000
Epoch: 18/30 Iteration: 572 Training loss: 0.00004
Epoch: 17/30 Iteration: 573 Validation Acc: 1.0000
Epoch: 18/30 Iteration: 573 Tra

Epoch: 22/30 Iteration: 700 Training loss: 0.00003
Epoch: 22/30 Iteration: 701 Training loss: 0.00003
Epoch: 21/30 Iteration: 702 Validation Acc: 1.0000
Epoch: 22/30 Iteration: 702 Training loss: 0.00001
Epoch: 22/30 Iteration: 703 Training loss: 0.00006
Epoch: 23/30 Iteration: 704 Training loss: 0.00000
Epoch: 22/30 Iteration: 705 Validation Acc: 1.0000
Epoch: 23/30 Iteration: 705 Training loss: 0.00007
Epoch: 23/30 Iteration: 706 Training loss: 0.00001
Epoch: 23/30 Iteration: 707 Training loss: 0.00001
Epoch: 22/30 Iteration: 708 Validation Acc: 1.0000
Epoch: 23/30 Iteration: 708 Training loss: 0.00001
Epoch: 23/30 Iteration: 709 Training loss: 0.00000
Epoch: 23/30 Iteration: 710 Training loss: 0.00001
Epoch: 22/30 Iteration: 711 Validation Acc: 1.0000
Epoch: 23/30 Iteration: 711 Training loss: 0.00002
Epoch: 23/30 Iteration: 712 Training loss: 0.00000
Epoch: 23/30 Iteration: 713 Training loss: 0.00000
Epoch: 22/30 Iteration: 714 Validation Acc: 1.0000
Epoch: 23/30 Iteration: 714 Tra

Epoch: 27/30 Iteration: 839 Training loss: 0.00002
Epoch: 26/30 Iteration: 840 Validation Acc: 1.0000
Epoch: 27/30 Iteration: 840 Training loss: 0.00000
Epoch: 27/30 Iteration: 841 Training loss: 0.00000
Epoch: 27/30 Iteration: 842 Training loss: 0.00000
Epoch: 26/30 Iteration: 843 Validation Acc: 1.0000
Epoch: 27/30 Iteration: 843 Training loss: 0.00000
Epoch: 27/30 Iteration: 844 Training loss: 0.00000
Epoch: 27/30 Iteration: 845 Training loss: 0.00000
Epoch: 26/30 Iteration: 846 Validation Acc: 1.0000
Epoch: 27/30 Iteration: 846 Training loss: 0.00003
Epoch: 27/30 Iteration: 847 Training loss: 0.00003
Epoch: 27/30 Iteration: 848 Training loss: 0.00000
Epoch: 26/30 Iteration: 849 Validation Acc: 1.0000
Epoch: 27/30 Iteration: 849 Training loss: 0.00000
Epoch: 27/30 Iteration: 850 Training loss: 0.00002
Epoch: 27/30 Iteration: 851 Training loss: 0.00000
Epoch: 26/30 Iteration: 852 Validation Acc: 1.0000
Epoch: 27/30 Iteration: 852 Training loss: 0.00000
Epoch: 27/30 Iteration: 853 Tra

inputs_fea.graph =   <tensorflow.python.framework.ops.Graph object at 0x7f54b7e63978>
INFO:tensorflow:Froze 4 variables.
Converted 4 variables to const ops.
15 ops in the final graph.
Tensor("predicted:0", shape=(?, 3), dtype=float32)


In [16]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    feed = {inputs_fea: test_x,
            labels_: test_y}
    test_acc = sess.run(accuracy, feed_dict=feed)
    print("Test accuracy: {:.4f}".format(test_acc))

INFO:tensorflow:Restoring parameters from checkpoints/zsy.ckpt
Test accuracy: 1.0000


In [17]:
# assert test_x.shape ==  (189, 4096)
# assert test_y.shape == (189, 3)

# from sklearn.metrics import confusion_matrix
# import seaborn as sns 
# import matplotlib.pyplot as plt

In [18]:
# cnf_matrix = confusion_matrix(y_true=test_y, y_pred=  ,labels=['jimaocai','qc','qingcai'])

# fig , ax = plt.subplots(1)
# ax = sns.heatmap(cnf_matrix, ax=ax, cmap=plt.cm.Blues, annot=True)
# ax.set_xticklabels(['jimaocai','qc','qingcai'])
# ax.set_yticklabels(['jimaocai','qc','qingcai'])
# plt.title('Confusion Matrix')
# plt.ylabel('True class')
# plt.xlabel('Predicted class')
# fig.set_size_inches(12, 8)
# plt.show()
# fig.savefig('confusion_matrix.png', dpi=100)

In [19]:
# with tf.Session() as _sess:
#   with gfile.FastGFile("/tmp/tfmodel/train.pbtxt",'rb') as f:
#     graph_def = tf.GraphDef()
#     graph_def.ParseFromString(f.read())
#     _sess.graph.as_default()
#     tf.import_graph_def(graph_def, name='tfgraph')